<a href="https://colab.research.google.com/github/jack-cao-623/python_learning/blob/main/pandas_merge_join_concat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas merging, joining, and concatenating
- combine 2 or more dataframes
- in Python Pandas:
  - join is only based on index
  - merge is more flexible and can combine using index or columns

In [1]:
# libraries needed
import numpy as np
import pandas as pd

In [2]:
# datasets we'll need
week1_sales = pd.read_csv(
    'https://raw.githubusercontent.com/jack-cao-623/python_learning/main/pandas/Restaurant%20-%20Week%201%20Sales.csv'
)

week2_sales = pd.read_csv(
    'https://raw.githubusercontent.com/jack-cao-623/python_learning/main/pandas/Restaurant%20-%20Week%202%20Sales.csv'
)

customers = pd.read_csv(
    'https://raw.githubusercontent.com/jack-cao-623/python_learning/main/pandas/Restaurant%20-%20Customers.csv'
)

foods = pd.read_csv(
    'https://raw.githubusercontent.com/jack-cao-623/python_learning/main/pandas/Restaurant%20-%20Foods.csv'
)

In [3]:
# examine data

# week1_sales is one row per order: which customer ordered which food in week 1
week1_sales.head()

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9


In [4]:
# week2_sales is one row per order: which customer ordered which food in week 2
week2_sales.head()

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


In [5]:
# one row per restaurant customer; 'ID' column matches to 'Customer ID' column in week1_sales and week2_sales
customers.head()

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive
3,4,Steven,Evans,Male,Fatz,Registered Nurse
4,5,Judy,Morrison,Female,Demivee,Legal Assistant


In [6]:
# foods is one row per food
foods.head()

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99
3,4,Quesadilla,4.25
4,5,Pizza,2.49


## pd.concat() method
- stack dataframes on top of each other
- like SQL UNION ALL
- syntax is pd.concat([df1, df2], ignore_index = True or False)
  - ignore_index = False keeps the same indices
  - ignore_index = True resets the index, i.e., a fresh one
- alternative syntax is df1.append(df2, ignore_index = True or False)

In [7]:
# week1_sales and week2_sales have same structure; stack on top of each other
pd.concat(
    objs = [week1_sales, week2_sales]     # week1_sales on top; week2_sales below
)

# note that there are 500 rows, but index is from 0-249
# this is because there are duplicated index values: 0-249 for week1_sales and 0-249 for week2_sales

# if you want an index from 0-499:
  # ignore_index = True

# see below for how this works

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,783,10
246,556,10
247,547,9
248,252,9


In [8]:
fake_dat_a = pd.DataFrame(data = [1, 2, 3], index = [0, 1, 2])
fake_dat_b = pd.DataFrame(data = [4, 5, 6], index = ['a', 'b', 'c'])

pd.concat(
    objs = [fake_dat_a, fake_dat_b],
    ignore_index = False              # False by default, which preserves original index in each dataframe; True resets the index
)

,0
0,1
1,2
2,3
a,4
b,5
c,6


In [9]:
# new sales dataframe with combines week1_sales and week2_sales
pd.concat(
    objs = [week1_sales, week2_sales],
    ignore_index = True                    # re-index, i.e., a new index
)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


In [10]:
# above is equivalent to:
(
    week1_sales
      .append(week2_sales, ignore_index = True)
)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


In [11]:
# multi-index such that we have unique indices and we can identify whether row came from week1_sales or week2_sales
sales = pd.concat(
    objs = [week1_sales, week2_sales],
    keys = ['week1', 'week2']                  # don't need to add indicator column manually
)

sales

# keys argument must be same length as objects
# these labels are the outermost layer
# innermost layer are original indices

Customer ID  Food ID
week1 0            537        9
      1             97        4
      2            658        1
      3            202        2
      4            155        9
...                ...      ...
week2 245          783       10
      246          556       10
      247          547        9
      248          252        9
      249          249        6

[500 rows x 2 columns]

## Inner Joins using .merge() method

In [12]:
# customers who ordered in week 1 and week 2

In [14]:
week1_sales.head()

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9


In [15]:
week2_sales.head()

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


In [19]:
# customers who ordered in week 1 and week 2
(
    week1_sales
      .merge(
          right = week2_sales,
          how = 'inner', 
          left_on = 'Customer ID',
          right_on = 'Customer ID'
      )
)

# Food ID_x refers to food ordered in week 1
# Food ID_y refers to food ordered in week 2
# Customer ID 155 is repeated
# Customer 155 ordered foods 9 and 1 in week 1; ordered food 3 in week 2

,Customer ID,Food ID_x,Food ID_y
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8
4,503,5,9
...,...,...,...
57,945,5,4
58,343,3,5
59,343,3,2
60,343,3,7


In [20]:
# customer 155 in week 1
week1_sales[week1_sales['Customer ID'] == 155]

,Customer ID,Food ID
4,155,9
17,155,1


In [22]:
# customer 155 in week 2
week2_sales[week2_sales['Customer ID'] == 155]

,Customer ID,Food ID
208,155,3


In [25]:
# see join in action for customer 155
(
week1_sales[week1_sales['Customer ID'] == 155]                     # left table
  .merge(
      right = week2_sales[week2_sales['Customer ID'] == 155],      # right table
      how = 'inner',                                               # inner join
      left_on = 'Customer ID',                                     # join on this key from left table
      right_on = 'Customer ID'                                     # join on this key from right table
  )
)

,Customer ID,Food ID_x,Food ID_y
0,155,9,3
1,155,1,3


In [28]:
# rename suffixes
(
    week1_sales                                  # left table
      .merge( 
          right = week1_sales,                   # right table
          how = 'inner',                         # inner join left and right tables
          left_on = 'Customer ID',               # key for left table
          right_on = 'Customer ID',              # key for right table
          suffixes = ['_week1', '_week2']        # suffixes; default is '_x' and '_y'
      )
)

,Customer ID,Food ID_week1,Food ID_week2
0,537,9,9
1,97,4,4
2,658,1,1
3,202,2,2
4,155,9,9
...,...,...,...
313,413,9,9
314,926,6,6
315,134,3,3
316,396,6,6
